In [1]:
# This will reconstruct the calibration and testing sub scans (test1, test2, and bones) for the DECT scans

import os
import numpy as np
from reconstruction.ring_corr import devon_correction
from reconstruction.find_dead_pixels import find_dead_pixels
from reconstruction.inidividual_bin_recon import correct_dead_pixels, reconstruct_CT
from scipy.interpolate import interp1d
from scipy.signal import medfilt

directory = r'\Material_decomposition_data/Calibration_and_test_data'
folders = ['23_08_26_CT_inserts_80', '23_08_26_CT_inserts_PCD']
peak_energies = [80, 120]
sub_folders = ['water_calib', 'water_test', 'opt1_calib', 'opt1_test', 'opt2_calib', 'opt2_test', 'bones_calib', 'bones_test']

# VARIABLES TO CHANGE
save = True

num_angles = 720  # The number of projections
scan_duration = 180  # The length of the CT scan (s)
offset = -1

proj_time = scan_duration/num_angles  # The scale factor (the scan time per projection/angle)

for lidx, load_folder in enumerate(folders):

    # Load the raw flat field, dark field, and water phantom scans (Only the full counts bin)
    air60 = np.load(os.path.join(directory, load_folder, 'airscan_65s', 'Data', 'data.npy'))[1:, :, :, -1]
    air1 = np.sum(air60, axis=0) / (60 / proj_time)  # Sum all frames to get the full 60s flat field scan (scaled correctly per frame)
    dark = np.load(os.path.join(directory, load_folder, 'darkscan_60s', 'Data', 'data.npy'))[:, :, -1]
    water = np.load(os.path.join(directory, load_folder, 'water', 'Data', 'data.npy'))[1:, :, :, -1]
    water = np.sum(water, axis=0)  # Sum all frames

    # Go through each sub folder
    for sub in sub_folders:

        data = np.load(os.path.join(directory, load_folder, sub, 'Data', 'data.npy'))[:, :, :, -1]

         # This will cut the projections down to the correct number if there are more than necessary
        if num_angles != len(data):
            diff = abs(num_angles - len(data))
            data = data[int(np.ceil(diff / 2)):len(data) - diff // 2]

        save_file = f'25-{peak_energies[lidx]}.npy'

        os.makedirs(os.path.join(directory, load_folder, sub, 'CT'), exist_ok=True)

        # The save locations for the sinogram, CT, and normalized CT
        sino_save_path = os.path.join(directory, load_folder, sub, 'Data', save_file)
        ct_save_path = os.path.join(directory, load_folder, sub, 'CT', f'CT_{save_file}')

        print(sub, save_file)

        # Create the dead pixel mask
        dpm = find_dead_pixels(np.sum(air60[0:6], axis=0), np.sum(air60[6:], axis=0), dark)

        # Correct raw data and air data for dead pixels
        print('Correcting dead pixels in raw data')
        data = correct_dead_pixels(data, dpm, True, False)
        print('Correcting dead pixels in air scan')
        air1 = correct_dead_pixels(air1, dpm, True, True)

        # Do the -ln(I/I0) correction
        sino = np.squeeze(np.log(air1 + 0.01) - np.log(data + 0.01))

        air1 = np.squeeze(air1)

        # Correct for pixel non-uniformities (ring artifacts)
        corr_array = devon_correction(water, air1 * (60 / proj_time), dpm, num_bins=1)

        # Correct the sinogram
        sino = np.multiply(corr_array, sino)

        # Additional correction
        sum_corr = np.sum(sino, axis=0)
        sum_dpm = np.ones((24, 576))
        xpts = np.arange(576)
        for z in range(24):
            f = interp1d(xpts, medfilt(sum_corr[z], 21))

            diff = np.abs(f(xpts) - sum_corr[z])

            sum_dpm[z][diff > 10] = np.nan

        sino = np.squeeze(correct_dead_pixels(sino, sum_dpm, True, False))

        # Save the sinogram and display it
        if save:
            np.save(sino_save_path, sino)

        # Now reconstruct the CT scan
         # Reconstruct the sinogram
        ct = reconstruct_CT(sino, h_offset=offset)

        # Save the CT scan
        if save:
            np.save(ct_save_path, ct)

water_calib 25-80.npy
Correcting dead pixels in raw data
Correcting secondary nan coords: 37440 left

C:\Users\drich\PycharmProjects\ReconRevamp\lda\find_dead_pixels.py:20: RuntimeWarning: divide by zero encountered in log
  dpm = np.abs(np.log(data1) - np.log(data2)) * 100
C:\Users\drich\PycharmProjects\ReconRevamp\lda\find_dead_pixels.py:20: RuntimeWarning: invalid value encountered in subtract
  dpm = np.abs(np.log(data1) - np.log(data2)) * 100
C:\Users\drich\PycharmProjects\ReconRevamp\general_functions.py:213: RuntimeWarning: Mean of empty slice
  avg = np.nanmean(vals, axis=0)



After correction: 0 dead pixels left

Correcting dead pixels in air scan
Correcting secondary nan coords: 52 left
After correction: 0 dead pixels left

Correcting dead pixels in water scan
Correcting secondary nan coords: 52 left
After correction: 0 dead pixels left

Correcting secondary nan coords: 46800 left
After correction: 0 dead pixels left

Starting reconstruction using FDK
water_test 25-80.npy
Correcting dead pixels in raw data
Correcting secondary nan coords: 37440 left
After correction: 0 dead pixels left

Correcting dead pixels in air scan
Correcting secondary nan coords: 52 left
After correction: 0 dead pixels left

Correcting dead pixels in water scan
Correcting secondary nan coords: 52 left
After correction: 0 dead pixels left

Correcting secondary nan coords: 46800 left
After correction: 0 dead pixels left

Starting reconstruction using FDK
opt1_calib 25-80.npy
Correcting dead pixels in raw data
Correcting secondary nan coords: 37440 left
After correction: 0 dead pixels

In [8]:
## This cell will normalize the CT scans to HU

import os
import numpy as np
from reconstruction.inidividual_bin_recon import normalize_ct
from glob import glob

directory = r'\Material_decomposition_data/Calibration_and_test_data'
folders = ['23_08_26_CT_inserts_80', '23_08_26_CT_inserts_PCD']

# VARIABLES TO CHANGE
save = True

# The sub folder names for each of the 3 CT acquisitions for the 3 sets of tissue equivalent cylinders
sub_folder_groups = [['water_calib', 'opt1_calib', 'opt2_calib', 'bones_calib'], ['water_test', 'opt1_test', 'opt2_test', 'bones_test']]

# Go through each folder
for load_folder in folders:

    # Go through each sub folder
    for sub_group in sub_folder_groups:

        ct_files = glob(os.path.join(directory, load_folder, sub_group[0], 'CT', '*roll*'))

        for file in ct_files:
            # Set the file and only the file's name
            file = file.split('\\')[-1]
            water_value = 0

            for sidx, sub in enumerate(sub_group):

                # Set the path to load the CT scan
                load_path = os.path.join(directory, load_folder, sub, 'CT', file)
                # print(load_path)

                print(sub, file)

                os.makedirs(os.path.join(directory, load_folder, sub, 'Norm CT'), exist_ok=True)

                # The save locations for the CT
                norm_save_path = os.path.join(directory, load_folder, sub, 'Norm CT', f'Norm_{file}')

                # Load the CT scan
                ct = np.load(load_path)

                # If it's the water scan normalize and save the water value to normalize the other scans
                if sidx == 0:
                    water_mask = np.load(os.path.join(directory, load_folder, sub, 'water_mask.npy'))
                    water_value = np.nanmean(ct[8] * water_mask)
                    ct = normalize_ct(ct, water_mask, water_slice=8)
                else:  # Use the water norm value to normalize the other scans
                    ct = normalize_ct(ct, water_norm_val=water_value)

                # Save the CT scan
                if save:
                    np.save(norm_save_path, ct)

water_calib CT_25-80_roll_-1.npy
opt1_calib CT_25-80_roll_-1.npy
opt2_calib CT_25-80_roll_-1.npy
bones_calib CT_25-80_roll_-1.npy
water_calib CT_25-80_roll_-2.npy
opt1_calib CT_25-80_roll_-2.npy
opt2_calib CT_25-80_roll_-2.npy
bones_calib CT_25-80_roll_-2.npy
water_calib CT_25-80_roll_-3.npy
opt1_calib CT_25-80_roll_-3.npy
opt2_calib CT_25-80_roll_-3.npy
bones_calib CT_25-80_roll_-3.npy
water_calib CT_25-80_roll_-4.npy
opt1_calib CT_25-80_roll_-4.npy
opt2_calib CT_25-80_roll_-4.npy
bones_calib CT_25-80_roll_-4.npy
water_calib CT_25-80_roll_-5.npy
opt1_calib CT_25-80_roll_-5.npy
opt2_calib CT_25-80_roll_-5.npy
bones_calib CT_25-80_roll_-5.npy
water_calib CT_25-80_roll_0.npy
opt1_calib CT_25-80_roll_0.npy
opt2_calib CT_25-80_roll_0.npy
bones_calib CT_25-80_roll_0.npy
water_calib CT_25-80_roll_1.npy
opt1_calib CT_25-80_roll_1.npy
opt2_calib CT_25-80_roll_1.npy
bones_calib CT_25-80_roll_1.npy
water_calib CT_25-80_roll_2.npy
opt1_calib CT_25-80_roll_2.npy
opt2_calib CT_25-80_roll_2.npy
bone

In [2]:
# This will take all the normalized data and the tissue cylinder masks and calibrate the model for 80 and 120 kVp

import os
import numpy as np

from decomposition.bourque import solve_for_c, solve_for_b

from datetime import datetime

directory = r'\Material_decomposition_data/Calibration_and_test_data'
folder_low = '23_08_26_CT_inserts_80'
folder_high = '23_08_26_CT_inserts_PCD'

low_t1, low_t2 = 25, 80
high_t1, high_t2 = 25, 120
K, M = 7, 7

file_low = f'Norm_CT_{low_t1}-{low_t2}.npy'
file_high = f'Norm_CT_{high_t1}-{high_t2}.npy'

sub_folders = ['water_calib', 'opt1_calib', 'opt2_calib', 'bones_calib']

# The labels for each of the vials in the three subsets
sub_labels = {'water_calib': ['Water'],
              'opt1_calib': ['LV1 Liver', 'LN-450 Lung', 'BRN-SR2 Brain', 'SB3 Cortical Bone'],
              'opt2_calib': ['Solid Water', 'AP6 Adipose', 'LN-300 Lung', 'BR-12 Breast', 'CB2-50%'],
              'bones_calib': ['IB Inner Bone', 'B-200 Bone', 'CB2-30%']
              }

z_values = {
            'Water': 7.693,
            'LN-300 Lung': 7.796,
            'LN-450 Lung': 7.719,
            'BR-12 Breast': 7.011,
            'AP6 Adipose': 6.362,
            'Solid Water': 7.777,
            'LV1 Liver': 7.773,
            'BRN-SR2 Brain': 6.362,
            'IB Inner Bone': 10.666,
            'B-200 Bone': 10.515,
            'CB2-30%': 11.095,
            'CB2-50%': 12.499,
            'SB3 Cortical Bone': 13.562
            }

rho_values = {
    'Water': 1,
    'LN-300 Lung': 0.28,
    'LN-450 Lung': 0.465,
    'BR-12 Breast': 0.957,
    'AP6 Adipose': 0.937,
    'Solid Water': 0.987,
    'LV1 Liver': 1.077,
    'BRN-SR2 Brain': 1.049,
    'IB Inner Bone': 1.106,
    'B-200 Bone': 1.105,
    'CB2-30%': 1.274,
    'CB2-50%': 1.467,
    'SB3 Cortical Bone': 1.691
}

start = datetime.now().timestamp()
# Go through each of the calibration sub folders and get the calibration values
calib_z = []
calib_rho = []
low_hu_calib = []
high_hu_calib = []

for idx, calib_sub in enumerate(sub_folders):

    calib_labels = sub_labels[calib_sub]

    for label in calib_labels:
        calib_z.append(z_values[label])
        calib_rho.append(rho_values[label])

    # Load the calibration data
    low_data = np.load(os.path.join(directory, folder_low, calib_sub, 'Norm CT', file_low))
    high_data = np.load(os.path.join(directory, folder_high, calib_sub, 'Norm CT', file_high))

    # Load the masks for the cylinders
    masks = np.load(os.path.join(directory, folder_low, calib_sub, 'mat_decomp_masks.npy'))

    if len((np.shape(masks))) < 3:
        masks = np.expand_dims(masks, axis=0)

    # Load the appropriate calibration slices
    low_slice_folder = os.path.join(directory, folder_low, calib_sub, 'Slice info')
    high_slice_folder = os.path.join(directory, folder_high, calib_sub, 'Slice info')

    low_slices = np.load(os.path.join(low_slice_folder, f'CT_{low_t1}-{low_t2}.npy'))
    high_slices = np.load(os.path.join(high_slice_folder, f'CT_{high_t1}-{high_t2}.npy'))

    good_slices = np.intersect1d(low_slices, high_slices)

    num_slices = len(good_slices)

    for i, mask in enumerate(masks):

        temp_low_hu = np.zeros(num_slices)
        temp_high_hu = np.zeros(num_slices)

        for cidx, cz in enumerate(good_slices):
            temp_low_hu[cidx] = np.nanmean(low_data[cz] * mask)
            temp_high_hu[cidx] = np.nanmean(high_data[cz] * mask)

        low_hu_calib.append(np.mean(temp_low_hu))
        high_hu_calib.append(np.mean(temp_high_hu))

low_hu_calib = np.array(low_hu_calib)
high_hu_calib = np.array(high_hu_calib)

calib_path = os.path.join(directory, folder_low, 'Calibrated_values')
os.makedirs(calib_path, exist_ok=True)

for K in range(2, 8):
    K_file = f'C_{low_t1}-{low_t2}_{high_t1}-{high_t2}_K{K}.npy'

    c = solve_for_c(calib_z, low_hu_calib, high_hu_calib, K=K)
    print(c)
    if not c is None:
        np.save(os.path.join(calib_path, K_file), np.array([c]))
for M in range(2, 8):

    # Solve for b_low and b_high
    M_file = f'B_{low_t1}-{low_t2}_{high_t1}-{high_t2}_M{M}.npy'

    b_low = solve_for_b(calib_z, calib_rho, low_hu_calib, M=M)
    b_high = solve_for_b(calib_z, calib_rho, high_hu_calib, M=M)
    print(b_low, b_high)
    if not b_low is None or b_high is None:
        np.save(os.path.join(calib_path, M_file), np.array([b_low, b_high]))

print(f'{datetime.now().timestamp() - start:0.3f} s')
print()

[ 7.35033775 70.52887171]
[  7.34301742  63.72194769 104.27629829]
[   7.36188939   65.58429448   -9.66083253 1127.56965459]
[ 7.36085889e+00  6.56804610e+01 -2.28293705e-01  7.67959450e+02
  2.84261760e+03]
[ 7.51715534e+00  3.65367824e+01 -1.20938909e+03  1.09192330e+05
 -2.03556299e+06  1.15261908e+07]
[ 7.51722618e+00  3.90496255e+01 -1.38025099e+03  9.67845434e+04
 -1.30838666e+06  9.73636149e+04  5.67927945e+07]
[0.03557471 0.12995021] [0.36612154 0.08546334]
[ 0.99820994 -0.07668406  0.01033702] [ 1.05556994 -0.06253007  0.00740347]
[ 1.42375937e+00 -2.12140723e-01  2.41602163e-02 -4.54682559e-04] [ 1.21672081e+00 -1.13826008e-01  1.26381634e-02 -1.72183256e-04]
[-3.57543703e+00  1.98933909e+00 -3.29953738e-01  2.41958975e-02
 -6.27307900e-04] [-1.83522031e+00  1.23014735e+00 -2.03543569e-01  1.48766592e-02
 -3.82962905e-04]
[ 1.00427171e+01 -5.30178328e+00  1.19790745e+00 -1.32552700e-01
  7.25659646e-03 -1.55760970e-04] [ 6.27712126e+00 -3.11317809e+00  7.06604144e-01 -7.84985

C:\Users\drich\PycharmProjects\ReconRevamp\material_decomp\bourque_method\bourque.py:37: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  c, c_res, c_rank, c_s = np.linalg.lstsq(L, z_calib)
C:\Users\drich\PycharmProjects\ReconRevamp\material_decomp\bourque_method\bourque.py:133: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  b, b_res, b_rank, b_s = np.linalg.lstsq(F, u)


In [3]:
# This will take all of the normalized data, the tissue cylinder masks, and the calibrated values and calculate the Zeff and electron density maps for 80 and 120 kVp

import os
import numpy as np
from decomposition.bourque import solve_for_z, solve_for_rho
from datetime import datetime

directory = r'D:\OneDrive - University of Victoria\Research\LDA Data'

folder_low = '23_08_26_CT_inserts_80'
folder_high = '23_08_26_CT_inserts_PCD'

low_t1, low_t2 = 25, 80
high_t1, high_t2 = 25, 120

file_low = f'Norm_CT_{low_t1}-{low_t2}.npy'
file_high = f'Norm_CT_{high_t1}-{high_t2}.npy'

# The sub folder names for each of the 3 CT acquisitions for the 3 sets of tissue equivalent cylinders
sub_folders = ['water_test', 'opt1_test', 'opt2_test', 'bones_test']

calib_path = os.path.join(directory, folder_low, 'Calibrated_values')

for K in range(2, 8):
    c = np.load(os.path.join(calib_path, f'C_{low_t1}-{low_t2}_{high_t1}-{high_t2}_K{K}.npy'))[0]
    for M in range(2, 8):

        b_low, b_high = np.load(os.path.join(calib_path, f'B_{low_t1}-{low_t2}_{high_t1}-{high_t2}_M{M}.npy'))

        start = datetime.now().timestamp()

        last_hu_idx = 0
        last_real_idx = 0
        for sidx, sub in enumerate(sub_folders):

            low_data = np.load(os.path.join(directory, folder_low, sub, 'Norm CT', file_low))
            high_data = np.load(os.path.join(directory, folder_high, sub, 'Norm CT', file_high))

            # Flatten the data arrays
            low_data = np.array(low_data.flatten())
            high_data = np.array(high_data.flatten())

            # Calculate the Zeff and rho maps for the whole image set
            z_calc = solve_for_z(low_data, high_data, c)
            rho_calc = solve_for_rho(z_calc, low_data, high_data, b_low, b_high)

            # Reshape the data into the original format
            z_calc = np.reshape(z_calc, (24, 512, 512))
            rho_calc = np.reshape(rho_calc, (24, 512, 512))

            # Save the data in the appropriate location
            mat_decomp_path = os.path.join(directory, folder_low, sub, 'Decomposition')
            os.makedirs(mat_decomp_path, exist_ok=True)

            np.save(os.path.join(mat_decomp_path, f'Z_map_K{K}_M{M}.npy'), z_calc)
            np.save(os.path.join(mat_decomp_path, f'rho_map_K{K}_M{M}.npy'), rho_calc)

        print(datetime.now().timestamp() - start)

C:\Users\drich\PycharmProjects\ReconRevamp\material_decomp\bourque_method\bourque.py:95: RuntimeWarning: divide by zero encountered in true_divide
  L = np.divide(u_low - u_high, u_low + u_high)


5.723704099655151
4.781420946121216
5.343701124191284
6.512243032455444
7.4534571170806885
8.19321584701538


c:\users\drich\anaconda3\envs\pcd_ct_recon\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


7.853058099746704
5.953812837600708
6.429009914398193
7.570284843444824
10.189759969711304
10.607069969177246
7.042866945266724
8.023371934890747
8.318972826004028
9.613761186599731
10.483707189559937
11.195785999298096
8.25084114074707
12.215928792953491
9.32093596458435
9.804759979248047
10.849870920181274
11.358780860900879
7.949173927307129
12.013731956481934
9.914289951324463
10.680562019348145
11.010637044906616
12.219023942947388
8.435472011566162
9.995586156845093
10.677491188049316
11.56200385093689
11.921812057495117
12.287659883499146


In [8]:
# This will take the Z and rho maps and calculate the results

import os
import numpy as np
import pandas as pd


directory = r'\Material_decomposition_data/Calibration_and_test_data'

load_folder = '23_08_26_CT_inserts_80'
K, M = 2, 3
low_t1, low_t2 = 25, 80
high_t1, high_t2 = 25, 120
folder_low = '23_08_26_CT_inserts_80'
folder_high = '23_08_26_CT_inserts_PCD'

file_z = f'Z_map_K{K}_M{M}.npy'
file_rho = f'rho_map_K{K}_M{M}.npy'

# The sub folder names for each of the 3 CT acquisitions for the 3 sets of tissue equivalent cylinders
sub_folders = ['water_test', 'opt1_test', 'opt2_test', 'bones_test']

# The labels for each of the vials in the three subsets
sub_labels = {'water_test': ['Water'],
              'opt1_test': ['LV1 Liver', 'LN-450 Lung', 'BRN-SR2 Brain', 'SB3 Cortical Bone'],
              'opt2_test': ['Solid Water', 'AP6 Adipose', 'LN-300 Lung', 'BR-12 Breast', 'CB2-50%'],
              'bones_test': ['IB Inner Bone', 'B-200 Bone', 'CB2-30%']
              }

z_values = {
    'Water': 7.693,
    'LN-300 Lung': 7.796,
    'LN-450 Lung': 7.719,
    'BR-12 Breast': 7.011,
    'AP6 Adipose': 6.362,
    'Solid Water': 7.777,
    'LV1 Liver': 7.773,
    'BRN-SR2 Brain': 6.362,
    'IB Inner Bone': 10.666,
    'B-200 Bone': 10.515,
    'CB2-30%': 11.095,
    'CB2-50%': 12.499,
    'SB3 Cortical Bone': 13.562
}

rho_values = {
    'Water': 1,
    'LN-300 Lung': 0.28,
    'LN-450 Lung': 0.465,
    'BR-12 Breast': 0.957,
    'AP6 Adipose': 0.937,
    'Solid Water': 0.987,
    'LV1 Liver': 1.077,
    'BRN-SR2 Brain': 1.049,
    'IB Inner Bone': 1.106,
    'B-200 Bone': 1.105,
    'CB2-30%': 1.274,
    'CB2-50%': 1.467,
    'SB3 Cortical Bone': 1.691
}

z_dict = {}
rho_dict = {}

num_water_pixels = 0
num_other_pixels = 0

for sidx, sub in enumerate(sub_folders):

    masks = np.load(os.path.join(directory, load_folder, sub, 'mat_decomp_masks.npy'))

    if len((np.shape(masks))) < 3:
        masks = np.expand_dims(masks, axis=0)

    num_mask_pixels = len(masks[0][np.logical_not(np.isnan(masks[0]))])

    if sub == 'water_test':
        num_water_pixels = len(masks[0][np.logical_not(np.isnan(masks[0]))])
    else:
        num_other_pixels = len(masks[0][np.logical_not(np.isnan(masks[0]))])

    mat_decomp_path = os.path.join(directory, load_folder, sub, 'Decomposition')

    z_calc = np.load(os.path.join(mat_decomp_path, file_z))
    rho_calc = np.load(os.path.join(mat_decomp_path, file_rho))

    # Find the good slices that agree between the bins
    low_slices = np.load(os.path.join(directory, folder_low, sub, 'Slice info', f'CT_{low_t1}-{low_t2}.npy'))
    high_slices = np.load(os.path.join(directory, folder_high, sub, 'Slice info', f'CT_{high_t1}-{high_t2}.npy'))

    good_slices = np.intersect1d(low_slices, high_slices)
    curr_num_slices = len(good_slices)

    for midx, mask in enumerate(masks):

        mat_type = sub_labels[sub][midx]

        z_mask_array = np.zeros((curr_num_slices, num_mask_pixels))
        rho_mask_array = np.zeros((curr_num_slices, num_mask_pixels))

        for zidx, z in enumerate(good_slices):

            # Get only the values within the specified cylinder
            temp_z = z_calc[z] * mask
            temp_rho = rho_calc[z] * mask

            # Remove nan values
            temp_z = temp_z[np.logical_not(np.isnan(temp_z))]
            temp_rho = temp_rho[np.logical_not(np.isnan(temp_rho))]

            z_mask_array[zidx] = temp_z
            rho_mask_array[zidx] = temp_rho

        if mat_type == 'Water':
            if 'Water' in z_dict:
                z_dict[mat_type] = np.concatenate((z_dict[mat_type], z_mask_array.flatten()))
                rho_dict[mat_type] = np.concatenate((rho_dict[mat_type], rho_mask_array.flatten()))
            else:
                z_dict[mat_type] = z_mask_array.flatten()
                rho_dict[mat_type] = rho_mask_array.flatten()
        else:
            z_dict[mat_type] = z_mask_array.flatten()
            rho_dict[mat_type] = rho_mask_array.flatten()

eval_columns = ['Z RMSE', 'Z mean', 'Z std', 'rho RMSE', 'rho mean', 'rho std', 'Z rel error', 'Z rel error std', 'rho rel error', 'rho rel error std']
eval_results = np.zeros((13, len(eval_columns)))

# RMSE values
for ik, key in enumerate(list(z_dict.keys())):
    z_real = z_values[key]
    rho_real = rho_values[key]

    water_z = z_values['Water']
    water_rho = rho_values['Water']

    z_key = z_dict[key]
    rho_key = rho_dict[key]

    if key == 'Water':
        num_mask_pixels = num_water_pixels
    else:
        num_mask_pixels = num_other_pixels

    z_rmse = np.reshape(z_key, (len(z_key)//num_mask_pixels, num_mask_pixels))
    rho_rmse = np.reshape(rho_key, (len(rho_key)//num_mask_pixels, num_mask_pixels))

    z_rmse = np.mean(z_rmse, axis=1)
    rho_rmse = np.mean(rho_rmse, axis=1)

    eval_results[ik, 0] = np.sqrt(np.mean(np.square(((z_real - z_key) / z_real)))) * 100 # Z RMSE
    eval_results[ik, 1] = np.mean(z_key)  # Z mean
    eval_results[ik, 2] = np.std(z_key)  # Z std

    eval_results[ik, 3] = np.sqrt(np.mean(np.square(((rho_real - rho_key) / rho_real)))) * 100 # rho RMSE
    eval_results[ik, 4] = np.mean(rho_key)  # rho mean
    eval_results[ik, 5] = np.std(rho_key)  # rho std

    eval_results[ik, 6] = np.mean(np.abs(z_rmse - z_real) / water_z) * 100
    eval_results[ik, 7] = np.std(np.abs(z_rmse - z_real) / water_z) * 100
    eval_results[ik, 8] = np.mean(np.abs(rho_rmse - rho_real) / water_rho) * 100
    eval_results[ik, 9] = np.std(np.abs(rho_rmse - rho_real) / water_rho) * 100

eval_results = pd.DataFrame(eval_results, columns=eval_columns, index=list(z_dict.keys()))
eval_results.to_csv(os.path.join(directory, load_folder, f'testing_results_K{K}_M{M}_DECT_zkey.csv'))

In [6]:
# This will gather all the different results for K and M and find the best values for each

import os
import numpy as np
import pandas as pd

# Data folder information
directory = r'\Material_decomposition_data/Calibration_and_test_data'
folder = '23_08_26_CT_inserts_80'

opt_columns = ['K', 'M', 'Z RMSE', 'rho RMSE']
opt_results = np.zeros((36, len(opt_columns)))

i = 0
for K in range(2, 8):
    for M in range(2, 8):
        # Load the pandas dataframe
        data = pd.read_csv(os.path.join(directory, folder, f'testing_results_K{K}_M{M}_DECT.csv'))

        # The values of each of the cylinders
        z_rmse = data['Z RMSE'].values
        rho_rmse = data['rho RMSE'].values

        print(K, M, f'{np.mean(z_rmse):0.3f}', f'{np.mean(rho_rmse):0.3f}')
        opt_results[i] = [K, M, np.mean(z_rmse), np.mean(rho_rmse)]
        i += 1

opt_results = pd.DataFrame(opt_results, columns=opt_columns)
opt_results.to_csv(os.path.join(directory, folder, f'DECT_optimization_results.csv'))


2 2 1.755 13.037
2 3 1.755 1.979
2 4 1.755 2.427
2 5 1.755 5.679
2 6 1.755 12.205
2 7 1.755 10.486
3 2 2.447 23.224
3 3 2.447 1.780
3 4 2.447 2.120
3 5 2.447 13.658
3 6 2.447 3.187
3 7 2.447 223.763
4 2 1.941 5.204
4 3 1.941 1.884
4 4 1.941 2.302
4 5 1.941 15.875
4 6 1.941 5.108
4 7 1.941 16.947
5 2 2.277 21.922
5 3 2.277 1.825
5 4 2.277 2.234
5 5 2.277 13.847
5 6 2.277 5.065
5 7 2.277 5.180
6 2 117.878 16.067
6 3 117.878 4.537
6 4 117.878 5.212
6 5 117.878 21.606
6 6 117.878 16.877
6 7 117.878 60.274
7 2 160.785 18.847
7 3 160.785 4.378
7 4 160.785 5.056
7 5 160.785 218.541
7 6 160.785 12.980
7 7 160.785 23.501


In [2]:
# # Now go through the good slices and find the noise in the images

import os
import numpy as np

directory = r'\Material_decomposition_data/Calibration_and_test_data'
high_folder = '23_08_26_CT_inserts_PCD'
low_folder = '23_08_26_CT_inserts_80'

low_t1, low_t2 = 25, 80
high_t1, high_t2 = 25, 120
K, M = 2, 3

file_z = f'Z_map_K{K}_M{M}.npy'
file_rho = f'rho_map_K{K}_M{M}.npy'

file_low = f'Norm_CT_{low_t1}-{low_t2}.npy'
file_high = f'Norm_CT_{high_t1}-{high_t2}.npy'

sub = 'water_test'

mask = np.load(os.path.join(directory, low_folder, sub, 'mat_decomp_masks.npy'))
num_mask_pixels = len(mask[np.logical_not(np.isnan(mask))])

mat_decomp_path = os.path.join(directory, low_folder, sub, 'Decomposition')
low_ct_path = os.path.join(directory, low_folder, sub, 'Norm CT')
high_ct_path = os.path.join(directory, high_folder, sub, 'Norm CT')

z_calc = np.load(os.path.join(mat_decomp_path, file_z))
rho_calc = np.load(os.path.join(mat_decomp_path, file_rho))

low_ct = np.load(os.path.join(low_ct_path, file_low))
high_ct = np.load(os.path.join(high_ct_path, file_high))

# Find the good slices that agree between the bins
low_slice_folder = os.path.join(directory, low_folder, sub, 'Slice info')
high_slice_folder = os.path.join(directory, high_folder, sub, 'Slice info')

low_slices = np.load(os.path.join(low_slice_folder, f'CT_{low_t1}-{low_t2}.npy'))
high_slices = np.load(os.path.join(high_slice_folder, f'CT_{high_t1}-{high_t2}.npy'))

good_slices = np.intersect1d(low_slices, high_slices)
curr_num_slices = len(good_slices)

z_mask_array = np.zeros((curr_num_slices, num_mask_pixels))
rho_mask_array = np.zeros((curr_num_slices, num_mask_pixels))

low_mask_array = np.zeros((curr_num_slices, num_mask_pixels))
high_mask_array = np.zeros((curr_num_slices, num_mask_pixels))

for zidx, z in enumerate(good_slices):

    # Get only the values within the water cylinder
    temp_z = z_calc[z] * mask
    temp_rho = rho_calc[z] * mask
    temp_low = low_ct[z] * mask
    temp_high = high_ct[z] * mask

    # Remove nan values
    temp_z = temp_z[np.logical_not(np.isnan(temp_z))]
    temp_rho = temp_rho[np.logical_not(np.isnan(temp_rho))]

    temp_low = temp_low[np.logical_not(np.isnan(temp_low))]
    temp_high = temp_high[np.logical_not(np.isnan(temp_high))]

    z_mask_array[zidx] = temp_z
    rho_mask_array[zidx] = temp_rho

    low_mask_array[zidx] = temp_low
    high_mask_array[zidx] = temp_high

print(f'Low energy noise: {np.nanstd(low_mask_array)} HU')
print(f'High energy noise: {np.nanstd(high_mask_array)} HU')
print(f'Z noise: {np.nanstd(z_mask_array)} HU')
print(f'Rho noise: {np.nanstd(rho_mask_array)} HU')

Low energy noise: 18.628598297198096 HU
High energy noise: 13.382342873188573 HU
Z noise: 0.556040273763703 HU
Rho noise: 0.031535625805351776 HU
